In [166]:
import requests
import sqlite3
import os
import time
import hashlib
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from datetime import date
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from requests_html import HTMLSession


bdPath = path.join(os.getcwd(), "BD", "bd.db")
downPath = path.join(os.getcwd(), "Downloads", "")
diariosPath = path.join(os.getcwd(), "Diarios")

In [168]:
def getId(dados):
    idD = dados.find('a')['href'].replace('javascript:chamarCaptcha(', '').replace(',\'TSE\');', '')
    return idD

def getMD5(key, idD):
    md5 = buscaBDId(idD)
    if md5 == -1:
        urlDownload = 'http://inter03.tse.jus.br/sadJudDiarioDeJusticaConsulta/'
        options = Options()
        prefs = {'download.prompt_for_download': False,
        'download.default_directory': downPath,
        'download.directory_upgrage': True,
        'profile.default_content_settings.popups': 0,
        }
        options.add_experimental_option('prefs', prefs)
        driver = webdriver.Chrome(options = options)
        driver.get(urlDownload)

        jsModificado = '''
                       function chamarCaptcha(a,t){$.ajax({type:"POST",url:"captcha.do",success:function(r){var c;$.ajax({type:"POST",contentType:"application/x-www-form-urlencoded; charset=UTF-8",data:{resposta:c,pergunta:r},url:"captcha.do",success:function(e){$("#pergunta").attr("value",r),$("#id").attr("value",a),$("#tribunal").attr("value",t),$("#resposta").attr("value",c),$("#captchaValidacao").attr("value","ok"),$("#formDocumentoDownload")[0].submit()},error:function(a){alert("Error: "+a)}})}})};
                       '''

        jsModificado = jsModificado + 'chamarCaptcha('+idD+',"TSE");'
        driver.execute_script(jsModificado)
        
        diarioPath = path.join(os.getcwd(), "Downloads", "TSE-"+key.replace('/', '_')+".pdf")
        while not os.path.exists(diarioPath):
            time.sleep(1)

        if os.path.isfile(diarioPath):
            md5 = hashlib.md5(open(diarioPath,'rb').read()).hexdigest()
            driver.close()
    return md5

def toISO(data):
    data = data.split('/')
    if(len(data) < 3): return None
    
    return data[2] +'-'+ data[1] +'-'+ data[0]

def buscaBDId(idD):
    return -1
    bd = sqlite3.connect(bdPath) 
    cursor = bd.cursor()
    cursor.execute('SELECT diario.md5 FROM diario WHERE id='+idD)
    md5 = -1
    for linha in cursor.fetchall():
        print(linha)
        break
    bd.close()
    return md5

def getMD5List(data):
    data = date.fromisoformat(data)
    #if data < datetime.now()
        #result = buscaDB(data)
        #if result:
        #    return result
    
    urlBusca = 'http://inter03.tse.jus.br/sadJudDiarioDeJusticaConsulta/diarioTxt.do'
    parametros = {'action' : 'buscarDiarios', 'voDiarioSearch.tribunal' : 'TSE', 'page' : 'diarioPageTextualList.jsp', 
                  'voDiarioSearch.dataPubIni' : data.strftime("%d/%m/%Y"), 'voDiarioSearch.dataPubFim' : data.strftime("%d/%m/%Y")} 
    
    
    req = requests.get(urlBusca, params=parametros)
    if req.status_code != 200:
        return req.status_code
    
    soup = bs(req.content, 'html.parser')
    tabela = soup.find_all('tr')
    diarios = dict()
    
    
    #For que remove o header e footer da tabela
    for i in range(2, len(tabela)):
        td = tabela[i].find_all('td')
        
        if len(td) < 3: continue
        key = td[0].get_text()
        
        dataDiario = date.fromisoformat(toISO(td[1].get_text()))
        if dataDiario.day != data.day: continue
        
        idDiario = getId(td[2])
        md5Diario = getMD5(key, idDiario)
        diarios.setdefault(key, [str(dataDiario), md5Diario, idDiario])
    
    print(diarios) 

In [169]:
getMD5List('2019-02-18')

http://inter03.tse.jus.br/sadJudDiarioDeJusticaConsulta/diarioTxt.do?action=buscarDiarios&voDiarioSearch.tribunal=TSE&page=diarioPageTextualList.jsp&voDiarioSearch.dataPubIni=18%2F02%2F2019&voDiarioSearch.dataPubFim=18%2F02%2F2019
{'34/2019': ['2019-02-18', 'e236d6ec787ac3eb1740ea7b321770e7', '96902']}
